In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# read the data
data = pd.read_csv('diamonds_data.csv')
data = data.drop(data.columns[0],axis=1)
data.head(3)

,carat,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,...,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,cut,color,clarity,priceperpoint,carat_discrete
0,0.23,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,1,2,7,1,1
1,0.21,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,2,2,6,1,1
2,0.23,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,4,2,4,1,1


In [3]:
data.info()
data.shape 
data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 26 columns):
carat             53940 non-null float64
cut_Fair          53940 non-null int64
cut_Good          53940 non-null int64
cut_Ideal         53940 non-null int64
cut_Premium       53940 non-null int64
cut_Very Good     53940 non-null int64
color_D           53940 non-null int64
color_E           53940 non-null int64
color_F           53940 non-null int64
color_G           53940 non-null int64
color_H           53940 non-null int64
color_I           53940 non-null int64
color_J           53940 non-null int64
clarity_I1        53940 non-null int64
clarity_IF        53940 non-null int64
clarity_SI1       53940 non-null int64
clarity_SI2       53940 non-null int64
clarity_VS1       53940 non-null int64
clarity_VS2       53940 non-null int64
clarity_VVS1      53940 non-null int64
clarity_VVS2      53940 non-null int64
cut               53940 non-null int64
color             53940 non-nul

,carat,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,...,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,cut,color,clarity,priceperpoint,carat_discrete
0,0.23,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,1,2,7,1,1
1,0.21,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,2,2,6,1,1
2,0.23,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,4,2,4,1,1


In [4]:
# split
#cut data set into train and target data
from sklearn.model_selection import train_test_split
sampled_data=pd.concat([data[data['priceperpoint']==4].sample(7000),data[data['priceperpoint']==3].sample(7000),data[data['priceperpoint']==2].sample(7000),data[data['priceperpoint']==1].sample(7000)],ignore_index=True)

print(sampled_data['priceperpoint'].value_counts())
X=sampled_data.drop(['priceperpoint'], axis=1)
y=sampled_data['priceperpoint']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.15, random_state=0, stratify=None)
print("training \n",y_train.value_counts())
print("testing \n",y_test.value_counts())

4    7000
3    7000
2    7000
1    7000
Name: priceperpoint, dtype: int64
training 
 1    5968
2    5953
4    5947
3    5932
Name: priceperpoint, dtype: int64
testing 
 3    1068
4    1053
2    1047
1    1032
Name: priceperpoint, dtype: int64


In [5]:
from sklearn import ensemble, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
#forest_fit = forest.fit(X_train, y_train)
#joblib.dump(forest_fit, 'rf.model')
clf = MLPClassifier(solver='lbfgs', #lbfgs比sgd優
                    activation='relu',
                    alpha=9e-4, 
                    learning_rate_init=0.5,
                    max_iter=800,
                    hidden_layer_sizes=(100, 50),
                    # early_stopping=True,
                    random_state=1)
clf.fit(X_train, y_train) 
# 預測
train_y_predicted = clf.predict(X_train)
# 績效
train_accuracy = metrics.accuracy_score(y_train, train_y_predicted)
# 預測
test_y_predicted = clf.predict(X_test)
# 績效
test_accuracy = metrics.accuracy_score(y_test, test_y_predicted)

print('train_accuracy: '+str(train_accuracy))
print('test_accuracy: '+str(test_accuracy))

/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


train_accuracy: 0.8752941176470588
test_accuracy: 0.8807142857142857


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [6]:
#數值化cut, color, clarity+離散化carat
X = sampled_data[['carat_discrete','cut','color','clarity']]
y = sampled_data['priceperpoint']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.15, random_state=0, stratify=None)

In [7]:
from sklearn import ensemble, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
#forest_fit = forest.fit(X_train, y_train)
#joblib.dump(forest_fit, 'rf.model')
clf = MLPClassifier(solver='lbfgs', #lbfgs比sgd優
                    activation='relu',
                    alpha=9e-4, 
                    learning_rate_init=0.5,
                    max_iter=800,
                    hidden_layer_sizes=(100, 50),
                    # early_stopping=True,
                    random_state=1)
clf.fit(X_train, y_train) 
# 預測
train_y_predicted = clf.predict(X_train)
# 績效
train_accuracy = metrics.accuracy_score(y_train, train_y_predicted)
# 預測
test_y_predicted = clf.predict(X_test)
# 績效
test_accuracy = metrics.accuracy_score(y_test, test_y_predicted)

print('train_accuracy: '+str(train_accuracy))
print('test_accuracy: '+str(test_accuracy))

train_accuracy: 0.7302521008403361
test_accuracy: 0.7173809523809523


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
#離散carat+二值化cut,color,clarity
X = sampled_data.drop(['cut','color','clarity','carat','priceperpoint'], axis=1)
y = sampled_data['priceperpoint']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.15, random_state=0, stratify=None)

In [9]:
from sklearn import ensemble, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
clf = MLPClassifier(solver='lbfgs', #lbfgs比sgd優
                    activation='relu',
                    alpha=9e-4, 
                    learning_rate_init=0.5,
                    max_iter=800,
                    hidden_layer_sizes=(10, 2),
                    # early_stopping=True,
                    random_state=1)
clf.fit(X_train, y_train) 
# 預測
train_y_predicted = clf.predict(X_train)
# 績效
train_accuracy = metrics.accuracy_score(y_train, train_y_predicted)
# 預測
test_y_predicted = clf.predict(X_test)
# 績效
test_accuracy = metrics.accuracy_score(y_test, test_y_predicted)

print('train_accuracy: '+str(train_accuracy))
print('test_accuracy: '+str(test_accuracy))

train_accuracy: 0.2507563025210084
test_accuracy: 0.24571428571428572


In [10]:
#連續carat+二值化3c
X = sampled_data.drop(['cut','color','clarity','carat_discrete','priceperpoint'], axis=1)
y = sampled_data['priceperpoint']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.15, random_state=0, stratify=None)

In [11]:
from sklearn import ensemble, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
#forest_fit = forest.fit(X_train, y_train)
#joblib.dump(forest_fit, 'rf.model')
clf = MLPClassifier(solver='lbfgs', #lbfgs比sgd優
                    activation='relu',
                    alpha=9e-4, 
                    learning_rate_init=0.5,
                    max_iter=800,
                    hidden_layer_sizes=(100, 2),
                    # early_stopping=True,
                    random_state=1)
clf.fit(X_train, y_train) 
# 預測
train_y_predicted = clf.predict(X_train)
# 績效
train_accuracy = metrics.accuracy_score(y_train, train_y_predicted)
# 預測
test_y_predicted = clf.predict(X_test)
# 績效
test_accuracy = metrics.accuracy_score(y_test, test_y_predicted)

print('train_accuracy: '+str(train_accuracy))
print('test_accuracy: '+str(test_accuracy))

train_accuracy: 0.8785294117647059
test_accuracy: 0.8807142857142857


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [12]:
#連續carat+數值化3c
X = sampled_data[['carat','cut','color','clarity']]
y = sampled_data['priceperpoint']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.15, random_state=0, stratify=None)

In [13]:
from sklearn import ensemble, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
#forest_fit = forest.fit(X_train, y_train)
#joblib.dump(forest_fit, 'rf.model')
clf = MLPClassifier(solver='lbfgs', #lbfgs比sgd優
                    activation='relu',
                    alpha=9e-4, 
                    learning_rate_init=0.5,
                    max_iter=800,
                    hidden_layer_sizes=(100, 2),
                    # early_stopping=True,
                    random_state=1)
clf.fit(X_train, y_train) 
# 預測
train_y_predicted = clf.predict(X_train)
# 績效
train_accuracy = metrics.accuracy_score(y_train, train_y_predicted)
# 預測
test_y_predicted = clf.predict(X_test)
# 績效
test_accuracy = metrics.accuracy_score(y_test, test_y_predicted)

print('train_accuracy: '+str(train_accuracy))
print('test_accuracy: '+str(test_accuracy))

train_accuracy: 0.8510084033613445
test_accuracy: 0.8638095238095238


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,test_y_predicted)
pd.crosstab(y_test, test_y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,1,2,3,4,All
True,,,,,
1,924,108,0,0,1032
2,164,728,155,0,1047
3,1,189,738,140,1068
4,0,7,384,662,1053
All,1089,1032,1277,802,4200


In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test,test_y_predicted))

              precision    recall  f1-score   support

           1       0.85      0.90      0.87      1032
           2       0.71      0.70      0.70      1047
           3       0.58      0.69      0.63      1068
           4       0.83      0.63      0.71      1053

    accuracy                           0.73      4200
   macro avg       0.74      0.73      0.73      4200
weighted avg       0.74      0.73      0.73      4200

